In [1]:
{-# LANGUAGE TypeOperators, ParallelListComp, ScopedTypeVariables, Rank2Types, UnicodeSyntax #-}
:opt no-lint

In [2]:
import Data.VectorSpace
import Math.LinearMap.Category
import Prelude ()
import Control.Category.Constrained.Prelude
type ℝ = Double

In [3]:
import System.Random
import Control.Monad (replicateM)
randomPoints <- replicateM 100 $ randomRIO (0,10) :: IO [ℝ]
randomDeviations <- cycle <$> replicateM 165 `id` randomRIO (-0.1, 0.1) :: IO [ℝ]

In [4]:
mapM_ print (
 linearRegressionW euclideanNorm
    (\x -> lfun $ \(a,b) -> a*x + b)
 <$> [ [(0,1), (1,2), (2,3)]
     , [(-2,6), (0,2), (1,0)]
     , [(x, 14.3*x + 2.9 + δy) | x<-randomPoints
                               | δy<-randomDeviations]
     :: [(ℝ,ℝ)] ])

(1.0,1.0)
(-2.0,1.9999999999999996)
(14.30043273454546,2.902335291125837)

In [5]:
import Linear.V2 (V2(..))

mapM_ print (
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> a<.>^squareV v + b<.>^v + c)
 <$> [ [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
     ] )

((SymTensor {getSymmetricTensor = ex.⊗V2 (-2.713331492287807e-5) 0.49999168286288986 ^+^ ey.⊗V2 0.49999168286288986 (-7.795338426497267e-5)},V2 3.0006562070987 2.0013599416106445),3.9914182545917356)

In [6]:
import Data.List (sort)
newtype Take = Take (∀ a . Int -> [a] -> [a])
takeSome' <- do
   randomIndices <- replicateM 10000 $ randomRIO (0, 10000)
   return $ Take (\n xs -> let l = length xs
                               go (i₀:i₁:is) xs'
                                | (e:es, r) <- splitAt (i₁-i₀) xs'  = e : go (i₁:is) r
                               go (i₀:is) (x₀:xs')  = x₀ : go is xs'
                               go _ _ = []
                           in go (sort . take n $ (`mod`l)<$>randomIndices) xs)
takeSome :: Int -> [a] -> [a]
Take takeSome = takeSome'

In [7]:
mapM_ print (
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> a<.>^squareV v + b<.>^v + c)
 <$> [ takeSome 6 $ [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
     ] )

((SymTensor {getSymmetricTensor = ex.⊗V2 (-1.7290049956159237e-2) 0.5027204848637447 ^+^ ey.⊗V2 0.5027204848637447 3.681908803855549e-3},V2 3.1186021763332974 1.9477810839598533),3.9534446992038283)